Autograd: automatic differentiation
===================================
Central to all neural networks in PyTorch is the ``autograd`` package.
Let’s first briefly visit this, and we will then go to training our
first neural network.

The ``autograd`` package provides automatic differentiation for all operations
on Tensors. 

Let us see this in more simple terms with some examples.

Tensor
--------
``torch.Tensor`` is the central class of the package. If you set its attribute
``.requires_grad`` as ``True``, it starts to track all operations on it. When
at some point you have a scalar you can call ``.backward()`` on it and the gradients 
of that scalar wrt. all tensors that require_grad are computed automatically. 
The of the scalar wrt. a tensor will be accumulated into ``.grad`` attribute of the tensor.

To stop a tensor from tracking history, you can call ``.detach()`` to detach
it from the computation history, and to prevent future computation from being
tracked.

To prevent tracking history (and using memory), you can also wrap the code block
in ``with torch.no_grad():``. This can be particularly helpful when evaluating a
model because the model may have trainable parameters with `requires_grad=True`,
but for which we don't need the gradients.

There’s one more class which is very important for autograd
implementation - a ``Function``.

``Tensor`` and ``Function`` are interconnected and build up an acyclic
graph, that encodes a complete history of computation. Each tensor has
a ``.grad_fn`` attribute that references a ``Function`` that has created
the ``Tensor`` (except for Tensors created by the user - their
``grad_fn is None``).

If you want to compute the derivatives, you can call ``.backward()`` on
a ``Tensor``. If ``Tensor`` is a scalar (i.e. it holds a one element
data), you don’t need to specify any arguments to ``backward()``,
however if it has more elements, you need to specify a ``gradient``
argument that is a tensor of matching shape.

In [ ]:
import torch

Create a tensor and set requires_grad=True to track computation with it

In [ ]:
x = torch.ones(2, 2, requires_grad=True)

Do an operation of tensor:

In [ ]:
y = x + 2
y

``y`` was created as a result of an operation, so it has a ``grad_fn``.

In [ ]:
y.grad_fn

Do more operations on y

In [ ]:
z = y * y * 3
out = z.mean()

out

Gradients
---------
Let's backprop now
Because ``out`` contains a single scalar.

In [ ]:
out.backward()

print gradients d(out)/dx

In [ ]:
print(x.grad)

You should have got a matrix of ``4.5``.

Let’s call the ``out``
*Tensor* $o$.
We have that $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.
Therefore,

\begin{equation}
\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)
\end{equation}

hence

\begin{equation}
\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5
\end{equation}

You can do many crazy things with autograd!

In [ ]:
x0 = torch.randn(2, requires_grad=True)
c = torch.tensor([-0.4, 0.6])

x = x0
while x.data.norm() < 1000:
    x = torch.stack((
        x[0] ** 2 - x[1] ** 2,
        2 * x[0] * x[1]
    )) + c
     
print(x)

In [ ]:
diff = ((x - torch.tensor([0.25, 0.])) ** 2).sum()
diff.backward()

In [ ]:
x0.grad

You can also stop autograd from tracking history on Tensors
with ``.requires_grad``=True by wrapping the code block in
``with torch.no_grad():``

In [ ]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
	print((x ** 2).requires_grad)

**Read Later:**

Documentation of ``autograd`` and ``Function`` is at
http://pytorch.org/docs/autograd